# Загрузка модулей:

In [ ]:
import os
import sys

# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'  # Без GPU
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# Для работы в обоих контейнерах (dl_utils/docker и dl_utils/PreAnnotation):
sys.path += ['./src', './..']

In [ ]:
from utils import TimeIt
from gdinoal import GDino
from ul_utils import UltralyticsModel
from ipy_utils import show_video, path2link
from cvat import (
    dir2unlabeled_tasks, tasks_auto_annottation, tasks2preview, subtask2xml,
    autofix_subtask, hide_skipped_objects_in_subtask
)
from cvat_srv import CVATSRV, tqdm

# Индексация файлов для предразметки:

In [ ]:
# path = 'path/to/dir'               # Путь до папки
# path = 'path/to/file'              # Путь до файла
# path = ['list', 'to', 'files']     # Или список путей до файлов
path = './project'

tasks = dir2unlabeled_tasks(path, desc='Индексация ресурсов')

# Загрузка модели и установка параметров предразметки:

In [ ]:
undefined_label = 'undefined'

### Фиксируем Grounding DINO в качестве предразметчика:

In [ ]:
from cv_utils import NMS
from boxmot_utils import Tracker

img2df = GDino(
    '/home/user/models/groundingdino_swinb_cogcoor.pth',
    box_threshold=0.15,
    text_threshold=0.15,
    prompt2label={
        'UFO': 'ufo',
        'Plane': 'ufo',
        'Bird': 'ufo',
        'Airship': 'ufo',
        'Helicopter': 'ufo',
        'Boat': 'boat',
        'Speedboat': 'boat',
        'Motor boat': 'boat',
        'Sailboat': 'boat',
        'Cruiser': 'boat',
        'Ship': 'boat',
        'Steamboat': 'boat',

        'Object': 'object',
        'Anything': 'object',
        'Stuff': 'object',

        'Person': 'person',
    },
    postprocess_filters=[NMS(), Tracker()]
)

img2df.device

### Фиксируем модель из Ultralytics в качестве предразметчика:

In [ ]:
from labels import LabelsConvertor

lc = LabelsConvertor('COCO_labels.xlsx',
                     'COCO_outdoor_superlabels.xlsx',
                     main_dict='label2superlabel')

# Фиксируем YOLO-трекер в качестве предразметчика:
img2df = UltralyticsModel(
    # model='rtdetr-x.pt',
    # model='yolo11x-seg.pt',
    tracker='botsort.yaml',
    mode='preannotation',
    postprocess_filters=[lc.apply2objs]
)

### Выполняем предразметку:

In [ ]:
# Фильтры постобработки, необходимые при использовании трекинга:
postprocess_filters = [
    autofix_subtask,
    hide_skipped_objects_in_subtask,
]

# Сама предразметка:
with TimeIt('разметку'):
    annotated_tasks = tasks_auto_annottation(
        tasks,
        img2df,
        label=undefined_label,
        postprocess_filters=postprocess_filters,
    )

# Сборка превью:

In [ ]:
with TimeIt('сборку превью'):
    out = show_video(tasks2preview(annotated_tasks, fps=30), size=320)
out

# Экспорт в CVAT:

In [ ]:
# Соединение с сервером:
cvat_srv = CVATSRV()
print('Соединение с', str(cvat_srv), 'установлено.')

In [ ]:
# Определяем проект для сохранения задач:

# Имя проекта в CVAT:
proj_name = 'test_proj'

labels = set(dino.prompt2label.values()) | {undefined_label}

# Если проект с зажданным именем существует:
if proj_name in cvat_srv.keys():

    # Берём его:
    proj = cvat_srv[proj_name]

    # Убеждаемся, что все необходимые метки в проекте уже есть:
    proj_labels = {label['name'] for label in proj.labels()}
    diff_labels = labels - proj_labels
    if diff_labels:
        raise ValueError(f'В проекте нет следующих меток: {diff_labels}')

    print('Используется имеющийся проект.')

else:
    proj = cvat_srv.new(proj_name, labels=labels)
    print('Создан новый проект.')

In [ ]:
# Сама выгрузка:

xml_file = 'annotations.xml'
for task in tqdm(annotated_tasks, desc='Выгрузка размеченных задач'):

    assert len(task) == 1

    df, file, true_frames = task[0]

    subtask2xml((df, file, true_frames), xml_file)

    if isinstance(file, str):
        name = os.path.basename(os.path.splitext(file)[0])
    else:
        name = os.path.basename(os.path.dirname(file[0]))

    if name in proj.keys():
        raise ValueError(f'В проекте уже есть задача с именем "{name}"!')

    proj.new(name, file, annotation_file=xml_file)